In [ ]:
from data.graph_loader import GraphLoader
from data.gtfs_loader import GTFSLoader
from core.routing import RouteService
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx
import random as rd

def openMap(m):
    html = "map.html"
    m.save(html)

    import webbrowser
    webbrowser.open(html)

In [2]:
gtfs_loader = GTFSLoader()
transit_df = gtfs_loader.load_transit_dataframe("data/gtfs")
stops_df = gtfs_loader.load_stops_dataframe("data/gtfs", transit_df)



Loading transit DataFrame from data/gtfs\transit_df.pkl
Loading stops DataFrame from data/gtfs\stops_df.pkl


In [3]:
graph_loader = GraphLoader()
graph_walk = graph_loader.create_graph_walk("data/graphs/ZMG_walk.pkl", "data/osm/ZMG_enclosure_2km.geojson")
graph_transit = graph_loader.create_graph_transit("data/graphs/ZMG_transit.pkl", stops_df)

Loading walking graph from data/graphs/ZMG_walk.pkl
Loading transit graph from data/graphs/ZMG_transit_300.pkl
Loading transit graph from data/graphs/ZMG_transit_300.pkl


In [4]:
route_service = RouteService(graph_walk, graph_transit, stops_df, transit_df)

In [43]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geo")
location = geolocator.geocode("ITESO, Guadalajara")

print(location.latitude, location.longitude)

20.673564 -103.3583501


In [ ]:
import folium
from core.routing import point_from_text
from shapely.geometry import Point

src = point_from_text("ITESO, Guadalajara")
dst = point_from_text("Aeropuerto Internacional de Guadalajara, Guadalajara")

if src is None or dst is None:
    print("No posible to geocode the provided addresses")
    #choose random points if geocoding fails
    start = rd.choice(list(graph_walk.nodes))
    destination = rd.choice(list(graph_walk.nodes))

    src = Point(graph_walk.nodes[start]['x'], graph_walk.nodes[start]['y'])
    dst = Point(graph_walk.nodes[destination]['x'], graph_walk.nodes[destination]['y'])

try:
    route_service.route_combined(src, dst)
    total_time, m = route_service.route_combined(src, dst)

    folium.LayerControl().add_to(m)

    print("Total time (minutes):", round(sum(total_time)/60))

    openMap(m)
    
except Exception as e:
    print(f"Error to calculate route: {e}")

      shape_id route_id         trip_id                  trip_headsign  \
152     C27_r1      C27      C27_trip_1  Hospital Civil de Guadalajara   
289     C89_r2      C89      C89_trip_2                         Copala   
371  MP-A06_r2   MP-A06  MP-A06-MM08-LV             Periférico Oriente   
383  MP-T02_r2   MP-T02  MP-T02-MM20-LV                  Chapalita Inn   

                                        shape_geometry route_short_name  \
152  LINESTRING (-11491441.015 2347919.12, -1149144...              C27   
289  LINESTRING (-11510403.177 2358936.671, -115103...              C89   
371  LINESTRING (-11513808.551 2358824.197, -115138...           MP-A06   
383  LINESTRING (-11515723.914 2351185.241, -115157...           MP-T02   

                          route_long_name  route_type route_color  \
152                     Troncal C27 - 605           3     #4C8D2B   
289                     Troncal C89 - 162           3     #4C8D2B   
371  Alimentadora Mi Macro Periférico A06     